In [4]:
import pandas as pd
import numpy as np
import xarray as xr

In [5]:
df = pd.read_csv("wildfires.csv")
df.head(10)
df.shape

C:\Users\jepoisso\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (16,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(1880465, 19)

In [7]:
# try querying precipiation in 2005
from math import cos, asin, sqrt

def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

def find_nearest(lat, long):
    distances = hotels.apply(
        lambda row: dist(lat, long, row['lat'], row['lon']), 
        axis=1)
    return hotels.loc[distances.idxmin(), 'name']

query = df.iloc[0]
ds = xr.open_dataset("mnet/pr_2005.nc")
pr2005_df = ds.to_dataframe()

#distances = pr2005_dr.apply(
#    lambda row: distance(row[])
#)

#pr2005_df.head(2)

C:\Users\jepoisso\Anaconda3\lib\site-packages\xarray\conventions.py:492: SerializationWarning: variable 'precipitation_amount' has _Unsigned attribute but is not of integer type. Ignoring attribute.
  new_vars[k] = decode_cf_variable(


In [8]:
pr2005_df

precipitation_amount
crs day        lat       lon                              
3   2005-01-01 49.400000 -124.766667                   NaN
                         -124.725000                   NaN
                         -124.683333                   NaN
                         -124.641667                   NaN
                         -124.600000                   NaN
...                                                    ...
    2005-12-31 25.066667 -67.225000                    NaN
                         -67.183333                    NaN
                         -67.141667                    NaN
                         -67.100000                    NaN
                         -67.058333                    NaN

[295945650 rows x 1 columns]

In [9]:
query

OBJECTID                        1
FOD_ID                          1
FIRE_YEAR                    2005
DISCOVERY_DATE         2.4534e+06
DISCOVERY_DOY                  33
DISCOVERY_TIME               1300
STAT_CAUSE_CODE                 9
STAT_CAUSE_DESCR    Miscellaneous
CONT_DATE              2.4534e+06
CONT_DOY                       33
CONT_TIME                    1730
FIRE_SIZE                     0.1
FIRE_SIZE_CLASS                 A
LATITUDE                  40.0369
LONGITUDE                -121.006
STATE                          CA
COUNTY                         63
FIPS_CODE                      63
FIPS_NAME                  Plumas
Name: 0, dtype: object

In [10]:
import datetime
import math
#type(query['FIRE_YEAR'])
q_date = datetime.datetime(query['FIRE_YEAR'], 1, 1) + datetime.timedelta(query['DISCOVERY_DOY'].astype(np.float64) - 1)
lat_min_max = (math.floor(query['LATITUDE']) - 1, math.ceil(query['LATITUDE']) + 1)
long_min_max = (math.floor(query['LONGITUDE']) - 1, math.ceil(query['LONGITUDE']) + 1)

lat_min_max, long_min_max

((39, 42), (-123, -120))

In [12]:
query

OBJECTID                        1
FOD_ID                          1
FIRE_YEAR                    2005
DISCOVERY_DATE         2.4534e+06
DISCOVERY_DOY                  33
DISCOVERY_TIME               1300
STAT_CAUSE_CODE                 9
STAT_CAUSE_DESCR    Miscellaneous
CONT_DATE              2.4534e+06
CONT_DOY                       33
CONT_TIME                    1730
FIRE_SIZE                     0.1
FIRE_SIZE_CLASS                 A
LATITUDE                  40.0369
LONGITUDE                -121.006
STATE                          CA
COUNTY                         63
FIPS_CODE                      63
FIPS_NAME                  Plumas
Name: 0, dtype: object

In [14]:
ds = xr.open_dataset("mnet/pr_2005.nc", decode_cf=False)
ds

<xarray.Dataset>
Dimensions:               (crs: 1, day: 365, lat: 585, lon: 1386)
Coordinates:
  * lon                   (lon) float64 -124.8 -124.7 -124.7 ... -67.1 -67.06
  * lat                   (lat) float64 49.4 49.36 49.32 ... 25.15 25.11 25.07
  * day                   (day) float64 3.835e+04 3.835e+04 ... 3.872e+04
  * crs                   (crs) uint16 3
Data variables:
    precipitation_amount  (day, lat, lon) uint16 ...
Attributes:
    geospatial_bounds_crs:      EPSG:4326
    Conventions:                CF-1.6
    geospatial_bounds:          POLYGON((-124.7666666333333 49.40000000000000...
    geospatial_lat_min:         25.066666666666666
    geospatial_lat_max:         49.40000000000000
    geospatial_lon_min:         -124.7666666333333
    geospatial_lon_max:         -67.058333300000015
    geospatial_lon_resolution:  0.041666666666666
    geospatial_lat_resolution:  0.041666666666666
    geospatial_lat_units:       decimal_degrees north
    geospatial_lon_units:       decimal_degrees east
    coordinate_system:          EPSG:4326
    author:                     John Abatzoglou - University of Idaho, jabatz...
    date:                       03 July 2019
    note1:                      The projection information for this file is: ...
    note2:                      Citation: Abatzoglou, J.T., 2013, Development...
    note3:                      Data in slices after last_permanent_slice (1-...
    note4:                      Data in slices after last_provisional_slice (...
    note5:                      Days correspond approximately to calendar day...

In [17]:
ds.sel(lon=-121.006, lat=40.0369, method="nearest")

<xarray.Dataset>
Dimensions:               (crs: 1, day: 365)
Coordinates:
    lon                   float64 -121.0
    lat                   float64 40.03
  * day                   (day) float64 3.835e+04 3.835e+04 ... 3.872e+04
  * crs                   (crs) uint16 3
Data variables:
    precipitation_amount  (day) uint16 ...
Attributes:
    geospatial_bounds_crs:      EPSG:4326
    Conventions:                CF-1.6
    geospatial_bounds:          POLYGON((-124.7666666333333 49.40000000000000...
    geospatial_lat_min:         25.066666666666666
    geospatial_lat_max:         49.40000000000000
    geospatial_lon_min:         -124.7666666333333
    geospatial_lon_max:         -67.058333300000015
    geospatial_lon_resolution:  0.041666666666666
    geospatial_lat_resolution:  0.041666666666666
    geospatial_lat_units:       decimal_degrees north
    geospatial_lon_units:       decimal_degrees east
    coordinate_system:          EPSG:4326
    author:                     John Abatzoglou - University of Idaho, jabatz...
    date:                       03 July 2019
    note1:                      The projection information for this file is: ...
    note2:                      Citation: Abatzoglou, J.T., 2013, Development...
    note3:                      Data in slices after last_permanent_slice (1-...
    note4:                      Data in slices after last_provisional_slice (...
    note5:                      Days correspond approximately to calendar day...

In [28]:
from datetime import datetime, timedelta
jan1st1900 = datetime(1900, 1, 1)
d = datetime(query['FIRE_YEAR'], 1, 1) + timedelta(days=np.float64(query['DISCOVERY_DOY']))
delta = d - jan1st1900

ds_prec = ds.sel(day=np.float64(delta.days), lon=-121.006, lat=40.0369, method="nearest")
ds_prec.to_dataframe()

,lon,lat,day,precipitation_amount
crs,,,,
3,-121.016667,40.025,38384.0,0


In [30]:
df

,OBJECTID,FOD_ID,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,CONT_DATE,CONT_DOY,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_CODE,FIPS_NAME
0,1,1,2005,2453403.5,33,1300.0,9.0,Miscellaneous,2453403.5,33.0,1730.0,0.10,A,40.036944,-121.005833,CA,63,63.0,Plumas
1,2,2,2004,2453137.5,133,845.0,1.0,Lightning,2453137.5,133.0,1530.0,0.25,A,38.933056,-120.404444,CA,61,61.0,Placer
2,3,3,2004,2453156.5,152,1921.0,5.0,Debris Burning,2453156.5,152.0,2024.0,0.10,A,38.984167,-120.735556,CA,17,17.0,El Dorado
3,4,4,2004,2453184.5,180,1600.0,1.0,Lightning,2453189.5,185.0,1400.0,0.10,A,38.559167,-119.913333,CA,3,3.0,Alpine
4,5,5,2004,2453184.5,180,1600.0,1.0,Lightning,2453189.5,185.0,1200.0,0.10,A,38.559167,-119.933056,CA,3,3.0,Alpine
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880460,1880461,300348363,2015,2457291.5,269,1726.0,13.0,Missing/Undefined,2457291.5,269.0,1843.0,0.01,A,40.481637,-122.389375,CA,NaN,NaN,NaN
1880461,1880462,300348373,2015,2457300.5,278,126.0,9.0,Miscellaneous,NaN,NaN,NaN,0.20,A,37.617619,-120.938570,CA,NaN,NaN,NaN
1880462,1880463,300348375,2015,2457144.5,122,2052.0,13.0,Missing/Undefined,NaN,NaN,NaN,0.10,A,37.617619,-120.938570,CA,NaN,NaN,NaN
1880463,1880464,300348377,2015,2457309.5,287,2309.0,13.0,Missing/Undefined,NaN,NaN,NaN,2.00,B,37.672235,-120.898356,CA,NaN,NaN,NaN


In [32]:
df[df['FIRE_YEAR'] == 2008]

,OBJECTID,FOD_ID,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,CONT_DATE,CONT_DOY,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_CODE,FIPS_NAME
5840,5841,5867,2008,2454564.5,99,1423.0,9.0,Miscellaneous,2454564.5,99.0,1435.0,0.10,A,44.566389,-83.993611,MI,135,135.0,Oscoda
26557,26558,26686,2008,2454471.5,6,1815.0,9.0,Miscellaneous,2454471.5,6.0,2030.0,50.00,C,36.823611,-91.097778,MO,35,35.0,Carter
26606,26607,26735,2008,2454473.5,8,1400.0,7.0,Arson,2454473.5,8.0,1430.0,0.10,A,36.412222,-82.192778,TN,163,163.0,Sullivan
26611,26612,26740,2008,2454466.5,1,1330.0,7.0,Arson,2454466.5,1.0,1730.0,0.25,A,34.931944,-76.841667,NC,49,49.0,Craven
26614,26615,26743,2008,2454466.5,1,1535.0,9.0,Miscellaneous,2454467.5,2.0,1400.0,0.10,A,30.985000,-89.053333,MS,111,111.0,Perry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1876976,1876977,300331400,2008,2454608.5,143,517.0,13.0,Missing/Undefined,NaN,NaN,NaN,4270.00,F,37.128611,-121.945556,CA,NaN,NaN,NaN
1877541,1877542,300334020,2008,2454638.5,173,1533.0,9.0,Miscellaneous,NaN,NaN,NaN,750.00,E,37.045614,-121.653211,CA,NaN,NaN,NaN
1878523,1878524,300338871,2008,2454628.5,163,1214.0,7.0,Arson,NaN,NaN,NaN,23344.00,G,39.762464,-121.736696,CA,NaN,NaN,NaN
1879498,1879499,300343761,2008,2454637.5,172,1358.0,2.0,Equipment Use,NaN,NaN,NaN,630.00,E,36.932050,-121.809000,CA,NaN,NaN,NaN


In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import xarray as xr
from IPython.display import display

df = pd.read_csv("wildfires.csv")
print("loaded df")

years = df['FIRE_YEAR'].unique()
mnet_datasets = [
    {
        'name': 'pdsi',
        'col': 'palmer_drought_severity_index',
        'ops': ['min', 'max', 'avg', 'count']
    }
]
jan1st1900 = datetime(1900, 1, 1)

def execute_op(op, entries_df):
    if op == 'min':
        return entries_df.min()
    elif op == 'max':
        return entries_df.max()
    elif op == 'avg':
        return entries_df.mean()
    elif op == 'count':
        return len(entries_df)
    return np.nan
        

for y in years:
    # go through each dataset per year
    for _, ds_metadata in enumerate(mnet_datasets):
        mnet_ds_filename = 'mnet/%s_%d.nc' % (ds_metadata['name'], y)
        mnet_ds = xr.open_dataset(mnet_ds_filename, decode_cf=False)
        time_start = datetime.now().timestamp()
        processed_count, invalid_count = 0, 0
        
        df_y = df[df['FIRE_YEAR'] == y]
        print("opened up %s" % mnet_ds_filename)
        dh = display('opened up %s' % mnet_ds_filename, display_id=True)
        # loop thru rows 
        for index, row in df_y.iterrows():
            #print("> processing index %d for year %d and dataset %s" % (index, y, mnet_ds_filename))
            # date from year + day of year
            f_year, disc_doy, cont_doy, latitude, longitude = query['FIRE_YEAR'], row['DISCOVERY_DOY'], row['CONT_DOY'], row['LATITUDE'], row['LONGITUDE']
            if np.isnan(f_year) or np.isnan(disc_doy) or np.isnan(latitude) or np.isnan(longitude):
                invalid_count += 1
                print("\tskipping invalid index %d" % (index), f_year, disc_doy, cont_doy, latitude, longitude)
                continue
                
            d_start = datetime(f_year, 1, 1) + timedelta(days=np.float64(disc_doy))
            d_end = datetime(f_year, 1, 1) + (timedelta(days=np.float64(cont_doy)) if not np.isnan(cont_doy) else timedelta(days=np.float64(disc_doy)))
            delta_start = (d_start - jan1st1900).days
            delta_end = (d_end - jan1st1900).days
            lat = np.float64(latitude)
            lon = np.float64(longitude)
            # select relevant entries in mnet_dataset
            mnet_entries = mnet_ds.sel(day=[delta_start, delta_end], lon=lon, lat=lat, method="nearest")
            mnet_df = mnet_entries.to_dataframe()
            # compute operations for entries and store in augment_cols
            augment_cols = dict()
            for op in ds_metadata['ops']:
                col_key = "%s_%s" % (ds_metadata['name'], op)
                augment_cols[col_key] = execute_op(op, mnet_df[ds_metadata['col']])
                # update dataset with new column
                df.at[index, col_key] = augment_cols[col_key]
            processed_count += 1    
            #print("\tentry %d : %d mnet entries in %s: %s" % (index, mnet_df.shape[0], mnet_ds_filename, augment_cols))
            if processed_count % 100 == 0:
                percentage = (processed_count + invalid_count) / df_y.shape[0]
                dh.update("[%d] '%s' dataset progress: %d / %d [%.2f] (%d invalids)" % (datetime.now().timestamp(), mnet_ds_filename, (processed_count + invalid_count), df_y.shape[0], percentage, invalid_count))
        time_end = datetime.now().timestamp()
        time_delta = time_end - time_start
        print("finished with %s, processed = %d, invalid = %d : took %d seconds" % (mnet_ds_filename, processed_count, invalid_count, time_delta))
    
output_f = "wildfire_augmented_%d.csv" % datetime.now().timestamp() 
print("done processing entries! output is %s" % output_f)
df.to_csv(output_f)

C:\Users\jepoisso\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (16,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


loaded df
opened up mnet/pdsi_2005.nc


"[1606553197] 'mnet/pdsi_2005.nc' dataset progress: 21100 / 88604 [0.24] (0 invalids)"